In [1]:
from pathlib import Path
import pandas as pd

PATH_NOIPS = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian\webxss\hardware_log_20251214_140109.csv")
PATH_IPS   = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian_model\ae\webxss\hardware_log_20251215_024027.csv")

RAM_TOTAL_MB = 8192


In [2]:
def extract_metrics(csv_path):
    df = pd.read_csv(csv_path)
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df["cpu_used_pct"] = 100 - df["cpu_idle_pct"]
    df["ram_used_mb"]  = RAM_TOTAL_MB - df["ram_available_mb"]
    df["disk_io_kbs"]  = df["disk_read_kbs"] + df["disk_write_kbs"]
    df["net_io_kbs"]   = df["net_rx_kbs"] + df["net_tx_kbs"]

    return {
        "cpu_mean": df["cpu_used_pct"].mean(),
        "cpu_max": df["cpu_used_pct"].max(),
        "ram_mean": df["ram_used_mb"].mean(),
        "ram_max": df["ram_used_mb"].max(),
        "disk_mean": df["disk_io_kbs"].mean(),
        "disk_max": df["disk_io_kbs"].max(),
        "net_mean": df["net_io_kbs"].mean(),
        "net_max": df["net_io_kbs"].max(),
    }


In [3]:
def process_single_csv(csv_path, scenario_name, attack_name):
    metrics = extract_metrics(csv_path)
    metrics["scenario"] = scenario_name
    metrics["attack"] = attack_name
    return pd.DataFrame([metrics])


In [4]:
noips_df = process_single_csv(
    PATH_NOIPS,
    scenario_name="no_ips",
    attack_name="webxss"
)

ips_df = process_single_csv(
    PATH_IPS,
    scenario_name="ips",
    attack_name="webxss"
)

all_df = pd.concat([noips_df, ips_df], ignore_index=True)


In [5]:
print(all_df.columns)
print(all_df)


Index(['cpu_mean', 'cpu_max', 'ram_mean', 'ram_max', 'disk_mean', 'disk_max',
       'net_mean', 'net_max', 'scenario', 'attack'],
      dtype='object')
    cpu_mean  cpu_max     ram_mean  ram_max   disk_mean  disk_max  net_mean  \
0   1.606351     3.18  2963.660676  2969.91   41.720811    668.90  1.235135   
1  16.843846    26.23  3232.380000  3292.00  875.327692  25176.25  2.084359   

   net_max scenario  attack  
0    11.82   no_ips  webxss  
1    13.79      ips  webxss  


In [6]:
mean_cols = ["cpu_mean", "ram_mean", "disk_mean", "net_mean"]

table_mean = (
    all_df
    .set_index(["attack", "scenario"])[mean_cols]
    .round(2)
)

print(table_mean)


                 cpu_mean  ram_mean  disk_mean  net_mean
attack scenario                                         
webxss no_ips        1.61   2963.66      41.72      1.24
       ips          16.84   3232.38     875.33      2.08


In [7]:
table_compare = (
    all_df
    .set_index(["attack", "scenario"])
    .sort_index()
    .round(2)
)

print(table_compare)


                 cpu_mean  cpu_max  ram_mean  ram_max  disk_mean  disk_max  \
attack scenario                                                              
webxss ips          16.84    26.23   3232.38  3292.00     875.33  25176.25   
       no_ips        1.61     3.18   2963.66  2969.91      41.72    668.90   

                 net_mean  net_max  
attack scenario                     
webxss ips           2.08    13.79  
       no_ips        1.24    11.82  


In [8]:
pivot = all_df.set_index("scenario")

delta_abs = (
    pivot.loc["ips", mean_cols] -
    pivot.loc["no_ips", mean_cols]
).to_frame(name="delta")

print(delta_abs.round(2))


                delta
cpu_mean    15.237495
ram_mean   268.719324
disk_mean  833.606881
net_mean     0.849224


In [9]:
delta_pct = (
    (pivot.loc["ips", mean_cols] - pivot.loc["no_ips", mean_cols])
    / pivot.loc["no_ips", mean_cols]
    * 100
).to_frame(name="delta_percent")

print(delta_pct.round(2))


          delta_percent
cpu_mean     948.577955
ram_mean       9.067142
disk_mean   1998.060117
net_mean      68.755541


In [10]:
import pandas as pd

# ===============================
# 1) Tabel perbandingan utama
# ===============================
table_compare = (
    all_df
    .set_index(["attack", "scenario"])
    .sort_index()
    .round(2)
)

table_compare_out = table_compare.reset_index()
table_compare_out["section"] = "table_compare"


# ===============================
# 2) Delta absolut (IPS - No IPS)
# ===============================
pivot = all_df.set_index("scenario")

delta_abs = (
    pivot.loc["ips", mean_cols] -
    pivot.loc["no_ips", mean_cols]
).round(2).to_frame(name="value")

delta_abs_out = delta_abs.reset_index().rename(columns={"index": "metric"})
delta_abs_out["section"] = "delta_absolute"


# ===============================
# 3) Delta persentase (%)
# ===============================
delta_pct = (
    (pivot.loc["ips", mean_cols] - pivot.loc["no_ips", mean_cols])
    / pivot.loc["no_ips", mean_cols]
    * 100
).round(2).to_frame(name="value")

delta_pct_out = delta_pct.reset_index().rename(columns={"index": "metric"})
delta_pct_out["section"] = "delta_percent"


# ===============================
# 4) Gabungkan semua
# ===============================
final_csv = pd.concat(
    [table_compare_out, delta_abs_out, delta_pct_out],
    ignore_index=True
)

# ===============================
# 5) Simpan ke 1 CSV
# ===============================
final_csv.to_csv("comparison_webxss.csv", index=False)

print("[INFO] comparison_webxss.csv saved")


[INFO] comparison_webxss.csv saved
